In [1]:
from pathlib import Path
from dotenv import load_dotenv
import os
import pandas as pd
from astropy import units as u
import numpy as np
from obsutils.utils import SkyportalClient
from galsynthspec.skyportal.query import strip_tns_name

In [2]:
from galsynthspec.paths import data_dir as galsynthspec_data_dir

In [3]:
uvot_data_dir = Path(os.getenv("UVOT_DATA_DIR"))

In [4]:
source = "AT2025iuh"
send_to_fritz = True
instrument_id = 29 # UVOT is 46

In [5]:
uvot_df = pd.read_csv(uvot_data_dir / f"{source}/uvot_summary.csv")
uvot_df

,Unnamed: 0,ISOT,MJD,RA,DEC,FILTER,EXPOSURE,AB_MAG,AB_MAG_ERR,AB_MAG_LIM,PARENT_DIR
0,0,2025-05-09T17:29:58.590,60804.729150,175.173875,18.238647,UVW2,920.78107,19.483274,0.089635,22.518467,19761001
1,1,2025-05-09T17:37:51.787,60804.734627,175.173875,18.238647,UVM2,920.80273,19.156223,0.081994,22.482860,19761001
2,2,2025-05-09T17:45:32.168,60804.739956,175.173875,18.238647,UVW1,865.28120,19.234810,0.076888,22.350693,19761001
3,3,2025-06-11T16:43:22.786,60837.696792,175.173875,18.238647,UVM2,499.75940,19.408901,0.109979,21.642593,19761002
4,4,2025-06-11T16:50:31.785,60837.701757,175.173875,18.238647,UVW1,333.39010,19.365845,0.108612,21.589762,19761002
5,5,2025-06-11T16:54:50.209,60837.704748,175.173875,18.238647,U,166.11954,19.456402,0.133981,20.981924,19761002
6,6,2025-06-11T17:01:41.163,60837.709504,175.173875,18.238647,UVW2,629.31700,19.615685,0.099921,22.475672,19761002


In [6]:
gss_df = pd.read_json(galsynthspec_data_dir / f"{source}/synthetic_photometry.json").set_index("band")
gss_df

,predicted_mag,sigma+,sigma-,measured_mag,measured_err,extinction,measured_mag_deextincted,predicted_mag_extincted
band,,,,,,,,
galex_FUV,21.896536,0.292748,0.284521,NaN,NaN,0.158460,NaN,22.054996
galex_NUV,21.672154,0.204190,0.233676,NaN,NaN,0.170105,NaN,21.842259
uvot_w2,21.820776,0.230559,0.248358,NaN,NaN,0.184202,NaN,22.004978
uvot_m2,21.692874,0.208514,0.237640,NaN,NaN,0.181583,NaN,21.874457
uvot_w1,21.535498,0.177685,0.208377,NaN,NaN,0.130447,NaN,21.665945
sdss_u0,21.044003,0.103820,0.112479,21.148690,0.139324,0.096482,21.052208,21.140485
sdss_g0,20.046372,0.056830,0.067209,20.115580,0.022473,0.075075,20.040505,20.121447
sdss_r0,19.275389,0.079840,0.110906,19.390750,0.020527,0.051429,19.339321,19.326818
sdss_i0,19.100986,0.064149,0.057349,19.049000,0.025018,0.038143,19.010857,19.139129


In [7]:
bands = [x.strip().lower() for x in uvot_df["FILTER"]]
uvot_band = [f"sdss_{x}0" if len(x) == 1 else f"uvot_{x[-2:]}" for x in bands]
mask = [x in ['sdss_u0', 'uvot_w1', 'uvot_m2', 'uvot_w2'] for x in uvot_band]
uvot_df["band"] = uvot_band
uvot_df = uvot_df[mask].reset_index(drop=True)
uvot_df

,Unnamed: 0,ISOT,MJD,RA,DEC,FILTER,EXPOSURE,AB_MAG,AB_MAG_ERR,AB_MAG_LIM,PARENT_DIR,band
0,0,2025-05-09T17:29:58.590,60804.729150,175.173875,18.238647,UVW2,920.78107,19.483274,0.089635,22.518467,19761001,uvot_w2
1,1,2025-05-09T17:37:51.787,60804.734627,175.173875,18.238647,UVM2,920.80273,19.156223,0.081994,22.482860,19761001,uvot_m2
2,2,2025-05-09T17:45:32.168,60804.739956,175.173875,18.238647,UVW1,865.28120,19.234810,0.076888,22.350693,19761001,uvot_w1
3,3,2025-06-11T16:43:22.786,60837.696792,175.173875,18.238647,UVM2,499.75940,19.408901,0.109979,21.642593,19761002,uvot_m2
4,4,2025-06-11T16:50:31.785,60837.701757,175.173875,18.238647,UVW1,333.39010,19.365845,0.108612,21.589762,19761002,uvot_w1
5,5,2025-06-11T16:54:50.209,60837.704748,175.173875,18.238647,U,166.11954,19.456402,0.133981,20.981924,19761002,sdss_u0
6,6,2025-06-11T17:01:41.163,60837.709504,175.173875,18.238647,UVW2,629.31700,19.615685,0.099921,22.475672,19761002,uvot_w2


In [8]:
hosts_df = pd.DataFrame(gss_df.loc[x] for x in uvot_df["band"]).reset_index(drop=True)
df = pd.concat([uvot_df, hosts_df], axis=1)
df

,Unnamed: 0,ISOT,MJD,RA,DEC,FILTER,EXPOSURE,AB_MAG,AB_MAG_ERR,AB_MAG_LIM,PARENT_DIR,band,predicted_mag,sigma+,sigma-,measured_mag,measured_err,extinction,measured_mag_deextincted,predicted_mag_extincted
0,0,2025-05-09T17:29:58.590,60804.729150,175.173875,18.238647,UVW2,920.78107,19.483274,0.089635,22.518467,19761001,uvot_w2,21.820776,0.230559,0.248358,NaN,NaN,0.184202,NaN,22.004978
1,1,2025-05-09T17:37:51.787,60804.734627,175.173875,18.238647,UVM2,920.80273,19.156223,0.081994,22.482860,19761001,uvot_m2,21.692874,0.208514,0.237640,NaN,NaN,0.181583,NaN,21.874457
2,2,2025-05-09T17:45:32.168,60804.739956,175.173875,18.238647,UVW1,865.28120,19.234810,0.076888,22.350693,19761001,uvot_w1,21.535498,0.177685,0.208377,NaN,NaN,0.130447,NaN,21.665945
3,3,2025-06-11T16:43:22.786,60837.696792,175.173875,18.238647,UVM2,499.75940,19.408901,0.109979,21.642593,19761002,uvot_m2,21.692874,0.208514,0.237640,NaN,NaN,0.181583,NaN,21.874457
4,4,2025-06-11T16:50:31.785,60837.701757,175.173875,18.238647,UVW1,333.39010,19.365845,0.108612,21.589762,19761002,uvot_w1,21.535498,0.177685,0.208377,NaN,NaN,0.130447,NaN,21.665945
5,5,2025-06-11T16:54:50.209,60837.704748,175.173875,18.238647,U,166.11954,19.456402,0.133981,20.981924,19761002,sdss_u0,21.044003,0.103820,0.112479,21.14869,0.139324,0.096482,21.052208,21.140485
6,6,2025-06-11T17:01:41.163,60837.709504,175.173875,18.238647,UVW2,629.31700,19.615685,0.099921,22.475672,19761002,uvot_w2,21.820776,0.230559,0.248358,NaN,NaN,0.184202,NaN,22.004978


In [9]:
mags = df["AB_MAG"].to_numpy() * u.ABmag
host_mags = df["predicted_mag_extincted"].to_numpy() * u.ABmag

In [10]:
subs = (mags.to(u.Jansky) - host_mags.to(u.Jansky)).to(u.ABmag)
subs

<Magnitude [19.59528269, 19.24886934, 19.35713214, 19.52719018,
            19.50490406, 19.7151072 , 19.74310629] mag(AB)>

In [11]:
df["sub_mag_deextincted"] = subs.value - df["extinction"]

In [12]:
sigma_p = ((df["predicted_mag_extincted"].to_numpy() + df["sigma+"].to_numpy()) * u.ABmag).to(u.Jansky)
sigma_p

<Quantity [4.63217449e-06, 5.33103817e-06, 6.64580880e-06, 5.33103817e-06,
           6.64580880e-06, 1.15419198e-05, 4.63217449e-06] Jy>

In [13]:
diff = host_mags.to(u.Jansky) - sigma_p
diff

<Quantity [1.09589951e-06, 1.12873452e-06, 1.18167122e-06, 1.12873452e-06,
           1.18167122e-06, 1.15814379e-06, 1.09589951e-06] Jy>

In [14]:
diff_uvot = ((df["AB_MAG"].to_numpy() - df["AB_MAG_ERR"].to_numpy()) * u.ABmag).to(u.Jansky) - mags.to(u.Jansky)
diff_uvot

<Quantity [5.02912563e-06, 6.19539201e-06, 5.39114687e-06, 6.67124923e-06,
           6.85042259e-06, 7.86738186e-06, 4.98651660e-06] Jy>

In [15]:
unc = (diff_uvot ** 2. + diff**2.)**0.5
unc

<Quantity [5.14714487e-06, 6.29737435e-06, 5.51913141e-06, 6.76606296e-06,
           6.95159237e-06, 7.95216916e-06, 5.10552089e-06] Jy>

In [16]:
err = (subs.to(u.ABmag) - (subs.to(u.Jansky) + unc).to(u.ABmag)).value

In [17]:
# df["sigma"] = 0.5 * (df["sigma+"] + df["sigma-"])
df["error"] = err

In [18]:
df[["ISOT", "FILTER", "AB_MAG", "AB_MAG_ERR", "sub_mag_deextincted", "error"]]

,ISOT,FILTER,AB_MAG,AB_MAG_ERR,sub_mag_deextincted,error
0,2025-05-09T17:29:58.590,UVW2,19.483274,0.089635,19.411080,0.101161
1,2025-05-09T17:37:51.787,UVM2,19.156223,0.081994,19.067287,0.090412
2,2025-05-09T17:45:32.168,UVW1,19.234810,0.076888,19.226685,0.087659
3,2025-06-11T16:43:22.786,UVM2,19.408901,0.109979,19.345607,0.123590
4,2025-06-11T16:50:31.785,UVW1,19.365845,0.108612,19.374457,0.124355
5,2025-06-11T16:54:50.209,U,19.456402,0.133981,19.618625,0.169046
6,2025-06-11T17:01:41.163,UVW2,19.615685,0.099921,19.558904,0.114275


In [19]:
ztf_name = source if source[:3] == "ZTF" else None
print(ztf_name)

None


In [20]:
client = SkyportalClient()
client.set_up_session()

In [21]:
def get_ztf_name(source):
    res = client.api(
        "GET", 
        f"sources/{strip_tns_name(source)}",
    )
    res.raise_for_status()
    ztf_name = [x["obj_id"] for x in res.json()["data"]["duplicates"] if x["obj_id"][:3] == "ZTF"][0]
    return ztf_name


In [22]:
# print(matches)

In [23]:
ORIGIN = "uvotredux/galsynthspec"

if send_to_fritz:

    payload = {
        "mjd": df["MJD"].to_list(),
        "filter": [f"uvot::{x.lower().strip()}" for x in df["FILTER"]],
        "obj_id": get_ztf_name(source),
        "instrument_id": instrument_id,
        "ra": df["RA"].to_list(),
        "dec": df["DEC"].to_list(),
        "ra_unc": None,
        "dec_unc": None,
        "origin": ORIGIN,
        "group_ids": [1],
        "magsys": "ab",
        "mag": df["sub_mag_deextincted"].to_list(),
        "magerr": df["error"].to_list(),
        "limiting_mag": df["AB_MAG_LIM"].to_list(),
        "limiting_mag_nsigma": 3,
    }
    print(payload)
    res = client.api(
        "PUT", 
        "photometry",
        data=payload
    )
    print(res.json())
    res.raise_for_status()

{'mjd': [60804.72915034919, 60804.73462715891, 60804.73995565, 60837.69679150903, 60837.701756768285, 60837.70474778704, 60837.7095042], 'filter': ['uvot::uvw2', 'uvot::uvm2', 'uvot::uvw1', 'uvot::uvm2', 'uvot::uvw1', 'uvot::u', 'uvot::uvw2'], 'obj_id': 'ZTF25aaooewz', 'instrument_id': 29, 'ra': [175.173875, 175.173875, 175.173875, 175.173875, 175.173875, 175.173875, 175.173875], 'dec': [18.2386472222222, 18.2386472222222, 18.2386472222222, 18.2386472222222, 18.2386472222222, 18.2386472222222, 18.2386472222222], 'ra_unc': None, 'dec_unc': None, 'origin': 'uvotredux/galsynthspec', 'group_ids': [1], 'magsys': 'ab', 'mag': [19.411080471501307, 19.06728662141176, 19.22668547215289, 19.345607460673925, 19.374457400653753, 19.618625123715766, 19.558904072544383], 'magerr': [0.10116130991627514, 0.09041152414489417, 0.08765931951990069, 0.12359017422935636, 0.12435463043188122, 0.1690460476066491, 0.11427505530151194], 'limiting_mag': [22.518467, 22.48286, 22.350693, 21.642593, 21.589762, 20.